# TII Falcon 7B Tutorial in HugginFace Transformers and Langchain 
In this tutorial I'm going to show how we can use the open-source Falcon 7B model developed by Technology Innovation Institute TII. Falcon supposedly outperforms GPT-3 for only 75% of the training compute budget whilst requiring a fifth of the compute at inference time. 


1. Installing packages

In [8]:
!pip install -q transformers 
!pip install -q accelerate einops 
!pip install -q langchain 
!pip install -q xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 15.1 MB/s eta 0:00:00


### Let's initialize a HuggingFace pipeline and load the LLM

If you're using Colab, make sure you use the GPU runtime, preferably the Pro membership as you might need access to either an A100 or V100 GPU.  
Running this tutorial locally will not be sufficient for most personal setups.  
The general steps that will be taken:  
1. Loading the Falcon model from HuggingFace 
2. Loading the model tokenizer 

In [2]:
from torch import cuda, bfloat16 
import transformers 
from transformers import AutoTokenizer, AutoModelForCausalLM 
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'  

#creating a model 
fmodel = AutoModelForCausalLM.from_pretrained(
    'tiiuae/falcon-7b-instruct',
    trust_remote_code=True,
    torch_dtype=bfloat16
)
fmodel.eval() 
fmodel.to(device) 
print(f'Model loaded on {device}') 

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


The tokenizer is loaded from the tii model itself, as implemented below. 

In [3]:
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b-instruct')

Initializing the HF pipeline and setting up some additional parameters 

In [9]:
generate_text = transformers.pipeline(
    model=fmodel, 
    tokenizer=tokenizer, 
    task='text-generation', 
    return_full_text=True, 
    device=device, 
    max_length=10000, 
    temperature=0.1, 
    top_p=0.15, #select from top tokens whose probability adds up to 15%
    top_k=0, #selecting from top 0 tokens 
    repetition_penalty=1.1, #without a penalty, output starts to repeat 
    do_sample=True, 
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [12]:
result = generate_text("What is the name of the first president of the united arab emirates?") 
print(result[0]['generated_text']) 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


What is the name of the first president of the united arab emirates?
The name of the first president of the United Arab Emirates is Sheikh Zayed bin Sultan Al Nahyan.


### LangChain Implementation 

In [30]:
from langchain import PromptTemplate, LLMChain, ConversationChain 
from langchain.chains.conversation.memory import (ConversationBufferMemory,ConversationSummaryMemory,ConversationBufferWindowMemory,ConversationKGMemory)
from langchain.llms import HuggingFacePipeline 
#Creating a basic prompt template 

template = """
I want you to act as an informative AI assistant. 
Answer the following question: {question}.
"""
prompt = PromptTemplate(
    input_variables=['question'], 
    template=template 
)

In [31]:
#showcasing the format of the prompt 
prompt.format(question="What is the name of the first US president? ")

'\nI want you to act as an informative AI assistant. \nAnswer the following question: What is the name of the first US president? .\n'

In [32]:
llm = HuggingFacePipeline(pipeline=generate_text)
llm_chain = LLMChain(llm=llm,prompt=prompt)

In [33]:
llm_chain.predict(question='generate a scala code that takes two integers as input and calculates the multiplcation and returns the answer as a floating point value') 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'def multiply(x: Int, y: Int): Double = {\n  x * y\n}'

### Implementing LangChain conversational memory

In [43]:
#creating a new conversational template 
template = """You are an informative assistant chatting with a human. 
{chat_history}
Human:{human_input}
Assistant:""" 
#creating the prompt 
prompt = PromptTemplate(
    input_variables=["chat_history","human_input"], 
    template=template
) 
memory = ConversationBufferMemory(memory_key="chat_history") 

In [44]:
llm = HuggingFacePipeline(pipeline=generate_text) 
#adding memory to the llm chain 
llm_chain = LLMChain(llm=llm,prompt=prompt,memory=memory) 

In [45]:
#1st prompt 
llm_chain.predict(human_input='What is the name of the first UAE president?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


' The name of the first UAE president is Zayed bin Sultan Al Nahyan.\nUser '

In [46]:
#2nd prompt 
llm_chain.predict(human_input='What is the name of the president that came after him?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


' The name of the president that came after Zayed bin Sultan Al Nahyan is His Highness Shaikh Khalifa bin Zayed Al Nahyan.\nUser '